# Exploration of Mixture Proportion Estimation using Kernel Embeddings

In [1]:
# -*- coding: utf-8 -*-
"""
DO MIXTURE PROPORTION ESTIMATION 
Using gradient thresholding of the $\C_S$-distance
"""
from cvxopt import matrix, solvers, spmatrix
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
plt.close('all')
import scipy.linalg as scilin
from Kernel_MPE_grad_threshold import *
import altair as alt
import pandas as pd
import time
np.random.seed(seed=12342)

In the cell below, I generate two datasets. The first is the mixture, composed of $(1-\kappa^*)$ proportion of observations with `size` number of covariates from the standard normal distribution and $\kappa^*$ proportion of observations with `size` number of covariates from a shifted standard normal distribution. The second dataset is the component, composed only of observations with covariates from the shifted standard normal distribution. For our problem, the mixture is the Iowa patient dataset while the component is the Wisconsin patient dataset.

After the data is generated, we call the MPE code using the two different thresholding techniques and print the results.

In [2]:
def MPE_sim(total_observations, kappa_star, size, mean_comp):
    """ Calls wrapper with a GMM as the mixture distribution and one of the 
    components as the component distribution. Replace the X_mixture and X_component 
    variables according to your data"""
    X_mixture = np.concatenate((np.random.randn(int((1-kappa_star)*total_observations/2), size), 
                                np.random.randn(int(kappa_star*total_observations/2), size)+
                                mean_comp))
    X_component = np.random.randn(int(total_observations/2), size) + mean_comp
    (KM1,KM2)=wrapper(X_mixture,X_component)
    return KM1,KM2
    
total_observations = 500
kappa_star = 0.4
size = 100
mean_comp = np.repeat(10.,size)
pred = MPE_sim(total_observations, kappa_star, size, mean_comp)
    
print("Kappa* = ",kappa_star)
print("Observations = ",total_observations)
print("Covariates = ",size)
print("KM1_estimate = ",pred[0])
print("KM2_estimate = ",pred[1])


ValueError: too many values to unpack (expected 2)

From the toy example, we see that when the target population is *much* different from the other component (henceforth called unrelated component) in the mixture, our algorithm does a very good job at approximating the true mixture proportion. 

However, what happens as we deviate from this simple case? Let's simulate the number of observations, the true mixture proportion, the number of covariates, and the separation between the target population and unrelated component. I will run 10,000 simulations and plot the results.

In [ ]:
#Create numpy array to store results
results = np.zeros((125,8))

#Run 1000 simulations for total_observation change
for i in range(0,125):
    if (i % 250) == 0:
        print("Simulation: ",i)
    results[i,0] = total_observations = 2*(i+5)
    results[i,1] = kappa_star = 0.4
    results[i,2] = size = 100
    mean_comp = np.repeat(10.,size)
    pred = MPE_sim(total_observations, kappa_star, size, mean_comp)
    results[i,3] = np.mean(mean_comp)
    results[i,4] = pred[0]
    results[i,5] = pred[1]    
    
#Calculate absolute error for each simulation, thresholding technique
results[:,6] = abs(results[:,1]-results[:,4])
results[:,7] = abs(results[:,1]-results[:,5])

#Generate a pandas dataframe
obs_results = pd.DataFrame(data = results, columns = ['total_observations', 'kappa_star', 'size','mean_comp','KM1','KM2','KM1_error','KM2_error'])

#Save results
obs_results.to_csv('results/obs_results.csv')

In [ ]:
#Create numpy array to store results
results = np.zeros((250,8))

#Create values for kappa
x = np.linspace(0.10,0.90,250)

#Run 1000 simulations for kappa_star change
for i in range(0,250):
    if (i % 250) == 0:
        print("Simulation: ",i)
    results[i,0] = total_observations = 500
    results[i,1] = kappa_star = x[i]
    results[i,2] = size = 100
    mean_comp = np.repeat(10.,size)
    pred = MPE_sim(total_observations, kappa_star, size, mean_comp)
    results[i,3] = np.mean(mean_comp)
    results[i,4] = pred[0]
    results[i,5] = pred[1]    
    
#Calculate absolute error for each simulation, thresholding technique
results[:,6] = abs(results[:,1]-results[:,4])
results[:,7] = abs(results[:,1]-results[:,5])

#Generate a pandas dataframe
kappa_results = pd.DataFrame(data = results, columns = ['total_observations', 'kappa_star', 'size','mean_comp','KM1','KM2','KM1_error','KM2_error'])

#Save results
kappa_results.to_csv('results/kappa_results.csv')

In [ ]:
#Create numpy array to store results
results = np.zeros((150,8))

#Run 1000 simulations for kappa_star change
for i in range(0,150):
    if (i % 125) == 0:
        print("Simulation: ",i)
    results[i,0] = total_observations = 500
    results[i,1] = kappa_star = 0.4
    results[i,2] = size = (i+2)
    mean_comp = np.repeat(10.,size)
    pred = MPE_sim(total_observations, kappa_star, size, mean_comp)
    results[i,3] = np.mean(mean_comp)
    results[i,4] = pred[0]
    results[i,5] = pred[1]    
    
#Calculate absolute error for each simulation, thresholding technique
results[:,6] = abs(results[:,1]-results[:,4])
results[:,7] = abs(results[:,1]-results[:,5])

#Generate a pandas dataframe
covariate_results = pd.DataFrame(data = results, columns = ['total_observations', 'kappa_star', 'size','mean_comp','KM1','KM2','KM1_error','KM2_error'])

#Save results
covariate_results.to_csv('results/covariate_results.csv')

In [ ]:
#Create numpy array to store results
results = np.zeros((250,8))
x = np.linspace(1,10,250)

#Run 1000 simulations for kappa_star change
for i in range(0,250):
    if (i % 250) == 0:
        print("Simulation: ",i)
    results[i,0] = total_observations = 500
    results[i,1] = kappa_star = 0.4
    results[i,2] = 100
    c = x[i]
    mean_comp = np.repeat(c,size)
    pred = MPE_sim(total_observations, kappa_star, size, mean_comp)
    results[i,3] = np.mean(mean_comp)
    results[i,4] = pred[0]
    results[i,5] = pred[1]    
    
#Calculate absolute error for each simulation, thresholding technique
results[:,6] = abs(results[:,1]-results[:,4])
results[:,7] = abs(results[:,1]-results[:,5])

#Generate a pandas dataframe
distance_results = pd.DataFrame(data = results, columns = ['total_observations', 'kappa_star', 'size','mean_comp','KM1','KM2','KM1_error','KM2_error'])

#Save results
distance_results.to_csv('results/distance_results.csv')

In [ ]:
#Visualize the results of our simulations
def make_plots(data, xvar, xlab):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3), sharey = True)
    axes[0].scatter(data[xvar],data['KM1_error'])
    axes[0].title.set_text('KM1')
    axes[1].scatter(data[xvar],data['KM2_error'], color = "Green")
    axes[1].title.set_text('KM2')
    axes[0].set(ylabel = 'Absolute Error')
    for ax in axes.flat:
        ax.set(xlabel=xlab)
    fig.tight_layout()
    return

make_plots(obs_results,'total_observations','Total Observations')
make_plots(kappa_results,'kappa_star','Kappa Star')
make_plots(covariate_results,'size','Number of Covariates')
make_plots(distance_results,'mean_comp','Mean of Component Distribution')

In [ ]:
#Runtime analysis for number of observations
kappa_star = 0.4
size = 100
mean_comp = np.repeat(10.,size)
results = np.zeros((45,2))
    
for i in range(0,45):
    results[i,0] = total_observations = 500+(i*100)
    start_time = time.time()
    pred = MPE_sim(total_observations, kappa_star, size, mean_comp)  
    results[i,1] = end_time = time.time() - start_time
    print("Nobs:",total_observations," Runtime: ", end_time)

In [ ]:
#Runtime analysis for number of covariates
kappa_star = 0.4
total_observations = 1000
results2 = np.zeros((45,2))
    
print("Covariates,Runtime")
for i in range(0,45):
    results2[i,0] = size = 100+(i*25)
    mean_comp = np.repeat(10.,size)
    start_time = time.time()
    pred = MPE_sim(total_observations, kappa_star, size, mean_comp)  
    results2[i,1] = end_time = time.time() - start_time
    print(size,",", end_time)

In [ ]:
#Save results
runtime_obs_results = pd.DataFrame(data = results, columns = ['total_observations', 'runtime'])
runtime_obs_results.to_csv('results/runtime_obs_results.csv')

In [ ]:
#Save results
runtime_cov_results = pd.DataFrame(data = results2, columns = ['size', 'runtime'])
runtime_cov_results.to_csv('results/runtime_cov_results.csv')

In [ ]:
#Plot results
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3), sharey = False)
axes[0].scatter(runtime_obs_results['total_observations'],runtime_obs_results['runtime'])
axes[0].title.set_text('Runtime vs. Total Observations')
axes[1].scatter(runtime_cov_results['size'],runtime_cov_results['runtime'], color = "Green")
axes[1].title.set_text('Runtime vs. Number of Covariates')
axes[0].set(ylabel = 'Runtime (s)', xlabel = 'Total Observations')
axes[1].set(xlabel = "Number of Covariates", ylim = (0,15))
fig.tight_layout()

In [4]:
#Identification studies
total_observations = 1000
kappa_star = 0.4
size = 100
nsim = 100
KM2_vec = np.zeros(nsim)
error_vec = np.zeros(nsim)

for sim in range(nsim):
    KM2_vec[sim], x = id_example(total_observations = total_observations
                                , kappa_star = kappa_star
                                , size = size)
    error_vec[sim] = float(sum(x[0] > ((1-kappa_star)*total_observations/2)))/(kappa_star*total_observations/2)
    
    
    

TypeError: 'float' object cannot be interpreted as an integer